### Visualize Chatbot/Response Data to Inform Business Decisions
#### Emily Flores: Week 2-7 project deliverables portfolio

Work on Customer Support Twitter dataset, modern corpus of tweets and replies. Check, analyze and understand the pattern and conversations, and for study of modern customer support practices and impact. Using the data, analyze and build interactive visualization to make business decision. 


####  Exploratory data analysis 

In [1]:
import pandas as pd

In [2]:
file_path = 'twcs.csv'
dataset = pd.read_csv('twcs.csv')

In [3]:
dataset.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
rows, columns = dataset.shape

In [5]:
print("Number of Rows:", rows)

Number of Rows: 2811774


In [6]:
print("Number of Columns:", columns)

Number of Columns: 7


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [8]:
description = dataset.describe()

In [9]:
print(description)

           tweet_id  in_response_to_tweet_id
count  2.811774e+06             2.017439e+06
mean   1.504565e+06             1.463141e+06
std    8.616450e+05             8.665730e+05
min    1.000000e+00             1.000000e+00
25%    7.601652e+05             7.155105e+05
50%    1.507772e+06             1.439805e+06
75%    2.253296e+06             2.220646e+06
max    2.987950e+06             2.987950e+06


In [10]:
dataset.isnull().sum()

tweet_id                         0
author_id                        0
inbound                          0
created_at                       0
text                             0
response_tweet_id          1040629
in_response_to_tweet_id     794335
dtype: int64

In [11]:
duplicateValues = dataset.duplicated()

In [12]:
print(duplicateValues.sum())

0


In [13]:
dataset[duplicateValues]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id


#### The data can be cleaned by reordering the 'created_at' attribute by date added.
#### Most recent tweet at the top of data set * oldest at the end.

In [14]:
dataset.sort_values(by='created_at', ascending=True, inplace=True)

In [15]:
dataset.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
2496505,2666295,751178,True,Fri Apr 01 17:37:48 +0000 2016,@AskSeagate Same issue Seagate. Is there a sol...,2666294,2666296.0
2496504,2666294,AskSeagate,False,Fri Apr 01 18:22:40 +0000 2016,@751178 Pls give us more clarification on your...,2666293,2666295.0
2496503,2666293,751178,True,Fri Apr 01 18:25:52 +0000 2016,@AskSeagate Bootstrap and media server daemon ...,2666291,2666294.0
2496501,2666291,AskSeagate,False,Fri Apr 01 19:53:33 +0000 2016,@751178 Pls use the link bellow to download th...,2666292,2666293.0
352823,402549,211231,True,Fri Apr 03 19:57:41 +0000 2015,@AsurionCares How do I get a receipt of my mon...,402547,NaN


#### The data can be cleaned by reorganizing the author_id by alphabetic order.
#### Start with 'author_id' that starts with 'a' & ends with 'z'

In [17]:
dataset.sort_values(by='author_id', ascending=True, inplace=True)

In [18]:
dataset.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
576082,648216,10026,True,Thu Oct 05 00:08:01 +0000 2017,Screw you @116016 and your stupid Blueprint pr...,648215,NaN
1543061,1693185,10026,True,Mon Nov 06 15:38:24 +0000 2017,"Dear @ChaseSupport, it’s kinda hard to pay my ...",1693184,NaN
1543060,1693184,10026,True,Mon Nov 06 15:45:29 +0000 2017,@ChaseSupport Actually it just doesn’t work in...,1693183,1693185.0
637146,713796,100363,True,Tue Oct 10 22:25:39 +0000 2017,Now the flight @Delta is sending our bag back ...,713795,NaN
1752883,1906927,10103,True,Wed Oct 25 09:42:58 +0000 2017,@MOO Big thanks to Quentin for the exceptional...,1906926,NaN


#### I noticed there are repeated 'author_id' we can group them together. 
#### While maintaing the 'author_id' in alphabetic order.
#### This will make it easier to access tweets from specific users. 

In [19]:
grouped_data = dataset.groupby('author_id')

In [20]:
dataset.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
576082,648216,10026,True,Thu Oct 05 00:08:01 +0000 2017,Screw you @116016 and your stupid Blueprint pr...,648215,NaN
1543061,1693185,10026,True,Mon Nov 06 15:38:24 +0000 2017,"Dear @ChaseSupport, it’s kinda hard to pay my ...",1693184,NaN
1543060,1693184,10026,True,Mon Nov 06 15:45:29 +0000 2017,@ChaseSupport Actually it just doesn’t work in...,1693183,1693185.0
637146,713796,100363,True,Tue Oct 10 22:25:39 +0000 2017,Now the flight @Delta is sending our bag back ...,713795,NaN
1752883,1906927,10103,True,Wed Oct 25 09:42:58 +0000 2017,@MOO Big thanks to Quentin for the exceptional...,1906926,NaN


In [21]:
inbound_count = dataset[dataset['inbound'] == True]['inbound'].count()

In [22]:
print("Number of inbound responses:", inbound_count)

Number of inbound responses: 1537843


In [23]:
outbound_count = dataset[dataset['inbound'] == False]['inbound'].count()

In [24]:
print("Number of outbound responses:", outbound_count)

Number of outbound responses: 1273931


In [25]:
unique_tweets_count = dataset['tweet_id'].nunique()

In [26]:
print("Number of unique tweets:", unique_tweets_count)

Number of unique tweets: 2811774


In [27]:
top_20_users = dataset['author_id'].value_counts().head(20)

In [28]:
print("Top 20 Twitter users:\n", top_20_users)

Top 20 Twitter users:
 AmazonHelp         169840
AppleSupport       106860
Uber_Support        56270
SpotifyCares        43265
Delta               42253
Tesco               38573
AmericanAir         36764
TMobileHelp         34317
comcastcares        33031
British_Airways     29361
SouthwestAir        28977
VirginTrains        27817
Ask_Spectrum        25860
XboxSupport         24557
sprintcare          22381
hulu_support        21872
sainsburys          19466
GWRHelp             19364
AskPlayStation      19098
ChipotleTweets      18749
Name: author_id, dtype: int64
